In [ ]:
## Load packages
import numpy as np
import geopandas as gpd
import pandas as pd
import xarray as xr
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds
from shapely.geometry import mapping
from shapely.geometry import Polygon


In [ ]:
## Define paths
# BA file for resampling resolution
path_wdpa = 'path-of-WDPA-shapefile'
path_ba = 'path-one-year-resampled-BA-file'

## output paths
output_tiff = 'output-path-to-save-as-tiff'
output_nc = 'output-path-to-save-as-netCDF'

In [12]:
## Read and clean WDPA data
gdf = gpd.read_file(path_wdpa)

# Ensure the STATUS_YR column is an integer
gdf['STATUS_YR'] = gdf['STATUS_YR'].astype(int)

# Remove areas with missing status year
gdf = gdf[gdf['STATUS_YR'] != 0] 

## Rasterize WDPA shapefile and create netCDF

In [13]:
# Read the burned area raster to get resolution and bounds
with rasterio.open(path_ba) as src:
    bounds = src.bounds  # Get bounds of the raster
    resolution = src.res  # Get resolution (tuple: (x_res, y_res))
    crs = src.crs  # Get CRS
    transform = src.transform  # Get affine transform

# Step 3: Create a blank raster
width = int((bounds.right - bounds.left) / resolution[0])  # Number of columns
height = int((bounds.top - bounds.bottom) / resolution[1])  # Number of rows
transform = from_bounds(bounds.left, bounds.bottom, bounds.right, bounds.top, width, height)

In [14]:
# Get WDPA shapes and rasterize based on resolution of BA
shapes = [(geom, getattr(row, 'STATUS_YR')) for geom, row in zip(gdf.geometry, gdf.itertuples())]

# Rasterize the shapes
raster = rasterize(
    shapes=shapes,
    out_shape=(height, width),
    transform=transform,
    fill=0,  # Value for pixels not covered by any polygon
    dtype='int32'
)

In [16]:
# Save rasterized output as tiff file
with rasterio.open(
    output_tiff,
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,
    dtype='int32',
    crs=crs,
    transform=transform,
) as dst:
    dst.write(raster, 1)

In [ ]:
## Load WDPA raster data 
with rasterio.open(output_tiff) as src:
    # Read raster data
    raster_data = src.read(1)  # Assuming the data is in the first band
    transform = src.transform
    crs = src.crs
    width = src.width
    height = src.height

In [ ]:
## Convert to netCDF

# Create PA_status and PA_year arrays
pa_status = np.where(raster_data > 0, 1, 0)  # 1 for protected, 0 for unprotected
pa_year = np.where(pa_status == 1, raster_data, np.nan)  # Year for protected, NaN for unprotected

# Create coordinates for the raster grid
x_coords = np.arange(width) * transform[0] + transform[2]  # Longitude values
y_coords = np.arange(height) * transform[4] + transform[5]  # Latitude values

# Create xarray Dataset
ds = xr.Dataset(
    {
        "PA_status": (["Latitude", "Longitude"], pa_status),
        "PA_year": (["Latitude", "Longitude"], pa_year),
    },
    coords={
        "Longitude": x_coords,
        "Latitude": y_coords,
    },
    attrs={
        "crs": str(crs),
        "description": "Protected Areas Dataset",
    }
)

# Save to NetCDF
ds.to_netcdf(output_nc)